In [1]:
#Vibrato tech task notebook
import pandas as pd
import numpy as np
import sqlite3
import csv

In [2]:
#url = "https://s3-ap-southeast-2.amazonaws.com/vibrato-data-test-public-datasets/world-food-facts.zip"
#df = pd.read_csv(url, compression = 'zip', sep='\t',skiprows=(0),header=(0))

In [3]:
foodTSV = 'en.openfoodfacts.org.products.tsv'

In [12]:
#open database file and csv reader (formatted for reading tsv)

SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
myCursor = conn.cursor() #provides connection to database

reader = csv.reader(open(foodTSV), delimiter='\t')
#print(reader.line_num)
#print(dir(reader))
headers = reader.__next__()
print(len(headers))

162


In [13]:
# Create table in sqlite3.  First 64 entries have type text, the rest as real numbers.  

do_this_sqlite_1 = "CREATE TABLE FoodFacts ( "

for headerName in headers[:64]:
    do_this_sqlite_1 += " " + headerName + " text,"

for headerName in headers[64:]:
    do_this_sqlite_1 += " " + headerName + " real,"
    
do_this_sqlite_1 = do_this_sqlite_1[:-1]

do_this_sqlite_1 += ");"

do_this_sqlite_1 = do_this_sqlite_1.replace('-', '_') #sql doesn't seem to like hyphens

print(do_this_sqlite_1)

myCursor.execute(do_this_sqlite_1)

CREATE TABLE FoodFacts (  code text, url text, creator text, created_t text, created_datetime text, last_modified_t text, last_modified_datetime text, product_name text, generic_name text, quantity text, packaging text, packaging_tags text, brands text, brands_tags text, categories text, categories_tags text, categories_en text, origins text, origins_tags text, manufacturing_places text, manufacturing_places_tags text, labels text, labels_tags text, labels_en text, emb_codes text, emb_codes_tags text, first_packaging_code_geo text, cities text, cities_tags text, purchase_places text, stores text, countries text, countries_tags text, countries_en text, ingredients_text text, allergens text, allergens_en text, traces text, traces_tags text, traces_en text, serving_size text, no_nutriments text, additives_n text, additives text, additives_tags text, additives_en text, ingredients_from_palm_oil_n text, ingredients_from_palm_oil text, ingredients_from_palm_oil_tags text, ingredients_that_ma

OperationalError: table FoodFacts already exists

In [14]:
#Insert values from tsv file into table

do_this_sqlite = "INSERT INTO FoodFacts VALUES ( "
do_this_sqlite += "?," * len(headers)
do_this_sqlite = do_this_sqlite[:-1]
do_this_sqlite += ")"

print(do_this_sqlite)

rowErrors = 0

for row in reader:
    try:
        myCursor.execute(do_this_sqlite,row)
    except:
        rowErrors += 1
        
print("rowErrors = {}".format(rowErrors))
    
conn.commit()

INSERT INTO FoodFacts VALUES ( ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
rowErrors = 48


In [10]:
SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
myCursor = conn.cursor() #provides connection to database
myCursor.execute("PRAGMA table_info(FoodFacts);")
results = myCursor.fetchall()
# print(results)
# conn.close()

In [16]:
SQLITE_FILE = "openFoodFacts.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
myCursor = conn.cursor() #provides connection to database
myCursor.execute("select product_name from FoodFacts limit 20;")
results = myCursor.fetchall()
print(results)
# conn.close()

[('Farine de blé noir',), ('Banana Chips Sweetened (Whole)',), ('Peanuts',), ('Organic Salted Nut Mix',), ('Organic Polenta',), ('Breadshop Honey Gone Nuts Granola',), ('Organic Long Grain White Rice',), ('Organic Muesli',), ('Organic Dark Chocolate Minis',), ('Organic Sunflower Oil',), ('Organic Adzuki Beans',), ('Organic Penne Pasta',), ('Zen Party Mix',), ('Organic Golden Flax Seeds',), ('Organic Spicy Punks',), ('Cinnamon Nut Granola',), ('Organic Hazelnuts',), ('Organic Sweetened Banana Chips',), ('Lotus Organic Brown Jasmine Rice',), ('Organic Oat Groats',)]
